In [31]:
import soft_clustering_dw
import l1_minimization
from utilities import get_wanted_file_paths, read_matrix

from pathlib import Path

import polars as pl
from tslearn.metrics import dtw
import numpy as np
import plotly.express as px

In [2]:
dtw_dir = Path("../processed_data/split_info/fold_0/fold_0/training/dtw")

In [3]:
dissimilarity_mat_path = Path(dtw_dir, "sex_female/age_80_to_84/dissimilarity_matrix.parquet.snappy")

In [24]:
dissimilarity_mat_df = pl.scan_parquet(dissimilarity_mat_path)
dissimilarity_mat_df = dissimilarity_mat_df.collect()
print(dissimilarity_mat_df)
dissimilarity_mat = dissimilarity_mat_df.to_numpy()

shape: (11, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ JS00167   ┆ JS07743   ┆ JS12994   ┆ JS17031   ┆ … ┆ JS22957   ┆ JS29031   ┆ JS43422   ┆ JS44357  │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.0       ┆ 18890.073 ┆ 20046.527 ┆ 16221.795 ┆ … ┆ 15036.605 ┆ 13577.965 ┆ 14121.617 ┆ 15748.89 │
│           ┆ 251       ┆ 379       ┆ 316       ┆   ┆ 858       ┆ 777       ┆ 974       ┆ 6094     │
│ 18890.073 ┆ 0.0       ┆ 20328.915 ┆ 17760.912 ┆ … ┆ 18892.622 ┆ 18188.600 ┆ 16823.799 ┆ 20058.63 │
│ 251       ┆           ┆ 538       ┆ 752       ┆   ┆ 274       ┆ 938       ┆ 281       ┆ 233      │
│ 20046.527 ┆ 20328.915 ┆ 0.0       ┆ 18509.523 ┆ … ┆ 19796.417 ┆ 20304.462

In [29]:
pl.Config.set_tbl_width_chars(200)
pl.Config.set_tbl_rows(34)
hf = pl.scan_parquet("../processed_data/all_header_file_data.parquet.snappy")
print(hf
    .filter(pl.col("patient_id").is_in(dissimilarity_mat_df.columns))
    .collect()
)



shape: (32, 4)
┌────────────┬─────┬────────┬───────────────────────────────────┐
│ patient_id ┆ age ┆ sex    ┆ snomed_concept_name               │
│ ---        ┆ --- ┆ ---    ┆ ---                               │
│ cat        ┆ u8  ┆ cat    ┆ cat                               │
╞════════════╪═════╪════════╪═══════════════════════════════════╡
│ JS20567    ┆ 80  ┆ Female ┆ Premature atrial contraction      │
│ JS20567    ┆ 80  ┆ Female ┆ Sinus bradycardia                 │
│ JS22957    ┆ 80  ┆ Female ┆ Prolonged QT interval             │
│ JS22957    ┆ 80  ┆ Female ┆ Sinus bradycardia                 │
│ JS44357    ┆ 84  ┆ Female ┆ Sinus tachycardia                 │
│ JS43422    ┆ 82  ┆ Female ┆ Prolonged QT interval             │
│ JS43422    ┆ 82  ┆ Female ┆ Sinus tachycardia                 │
│ JS43422    ┆ 82  ┆ Female ┆ Left ventricular hypertrophy      │
│ JS43422    ┆ 82  ┆ Female ┆ ST segment changes                │
│ JS29031    ┆ 81  ┆ Female ┆ Sinus bradycardia              

In [5]:
wfdb_dir = Path("../raw_data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords")
a_path, b_path = get_wanted_file_paths(
    parent_dir=wfdb_dir,
    file_stems=["JS23065", "JS23622"]
)
a = read_matrix(a_path)
b = read_matrix(b_path)

In [ ]:
D_hat, W = l1_minimization.iteratively_reweighted_l1_minimization(
    D=dissimilarity_mat
)

In [19]:
D_hat = np.round(D_hat)

In [33]:
soft_obj, mds = soft_clustering_dw.soft_assignment_gmm(
    dm=D_hat,
    n_clusters=2,
    n_dim=2
)

In [34]:
mds.shape

(11, 2)

In [36]:
fig = px.scatter(
    x=mds[:, 0],
    y=mds[:, 1]
)
fig.show()
fig.write_image(Path("../processed_data/split_info/fold_0/fold_0/training/dtw/sex_female/age_80_to_84/mds.png"))

In [39]:
soft_obj

array([[9.84638096e-01, 1.53619044e-02],
       [1.33379438e-03, 9.98666206e-01],
       [1.17929819e-02, 9.88207018e-01],
       [9.77773796e-01, 2.22262039e-02],
       [1.37232182e-02, 9.86276782e-01],
       [9.99998917e-01, 1.08316635e-06],
       [9.99999999e-01, 7.73669405e-10],
       [9.99999997e-01, 2.57151865e-09],
       [9.99968969e-01, 3.10312298e-05],
       [3.30642382e-01, 6.69357618e-01],
       [9.99999318e-01, 6.82175462e-07]])

In [37]:
fuzzy_c_means = soft_clustering_dw.soft_assignment_fuzzy(
    dm=D_hat,
    n_clusters=2
)

In [38]:
fuzzy_c_means

array([[5.21935075e-01, 4.78064925e-01],
       [5.01411694e-01, 4.98588306e-01],
       [4.84054583e-01, 5.15945417e-01],
       [4.97160592e-01, 5.02839408e-01],
       [5.13928466e-01, 4.86071534e-01],
       [4.87793081e-01, 5.12206919e-01],
       [7.31582413e-11, 1.00000000e+00],
       [5.07375790e-01, 4.92624210e-01],
       [1.00000000e+00, 7.31582413e-11],
       [5.09517335e-01, 4.90482665e-01],
       [5.15523875e-01, 4.84476125e-01]])